In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]=groq_api_key
model1=ChatGroq(model="gemma2-9b-it")
model2=ChatGroq(model="llama-3.1-8b-instant")

In [5]:
from langchain_core.prompts import PromptTemplate

# 1. Notes
prompt1=PromptTemplate(
    template="Generate short and simple notes on following text: {text}",
    input_variables=["text"],
)

# 2. Quiz
prompt2=PromptTemplate(
    template="Generate 5 short question answers from following text: {text}",
    input_variables=["text"],
)

# 3. Merge
prompt3=PromptTemplate(
    template="""Merge the following notes and quiz into single docs.
    Notes: {notes}
    Quiz: {quiz}
    """,
    input_variables=["notes","quiz"]
)

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

RunnableParallel => helps in running multiple chain together

In [8]:
from langchain.schema.runnable import RunnableParallel

In [10]:
# parallel chain
parallel_chain=RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz' : prompt2 | model2 | parser 
})

In [11]:
# final sequential chain
merge_chain = prompt3 | model2 | parser

chain = parallel_chain | merge_chain

In [12]:
text = """
Binary Search is a highly efficient searching algorithm used to locate a target element in a sorted array or list.
It follows the principle of divide and conquer, which means that instead of scanning each element sequentially, it repeatedly divides the search interval in half until the element is found or the interval becomes empty.
The process begins by comparing the target value with the middle element of the array.
If the target matches the middle element, its index is returned.
If the target is smaller, the algorithm continues searching in the left half; if larger, it searches in the right half.
This halving process ensures that with each step, the number of elements under consideration is reduced by half, making the search significantly faster than linear search for large datasets. Binary search has a best-case time complexity of O(1) when the target is found immediately at the middle, and an average and worst-case time complexity of O(log n), 
since the search space is halved at each iteration. 
The iterative implementation of binary search requires only constant auxiliary space, whereas the recursive implementation requires O(log n) space due to recursive function calls. 
While it is extremely efficient on sorted data, binary search cannot be applied to unsorted arrays without preprocessing. Despite this limitation, it remains one of the most fundamental and widely used algorithms in computer science due to its efficiency, simplicity, and predictable logarithmic performance.
"""

In [14]:
print(chain.invoke({"text": text}))

## Binary Search Notes and Quiz

**What it does:**

* Finds a target value in a *sorted* array.

**How it works:**

1. **Divide and Conquer:** Repeatedly divides the search interval in half.
2. **Comparison:** Compares the target value to the middle element.
3. **Narrowing:**
    * If target matches the middle, done!
    * If target is smaller, search the left half.
    * If target is larger, search the right half.

**Advantages:**

* **Very efficient:** Much faster than linear search for large datasets.
* **Logarithmic time complexity:**
    * Best case: O(1) (target at the middle)
    * Average & Worst case: O(log n)

**Disadvantages:**

* **Requires sorted data:** Can't be used on unsorted arrays without sorting first.

**Space Complexity:**

* Iterative: O(1) (constant space)
* Recursive: O(log n) (due to function calls)

**Key Points:**

* Fundamental algorithm in computer science.
* Widely used due to efficiency and predictable performance.

### Quiz

1. **What is the basic princ

In [15]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP